In [ ]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.firstpos = set()
        self.lastpos = set()
def augmentation(re):
    return re+".#"
def updateRegex(re):
    precedence = {"(": 0,")": 0,"|": 1, ".": 2, "*": 3 }
    stack = Stack()
    newRegx = ''
    for i in range(len(re)):
        if re[i].isalpha() or re[i] == "#":
            #print(re[i],"89")
            newRegx += re[i]
        else:
            if stack.isempty():
                stack.push(re[i])
                #print(stack.list)
            else:
                if re[i] == "(":
                    stack.push(re[i])
                elif re[i] == ")":
                    while stack.list[-1] != "(":
                        newRegx += stack.pop()
                    stack.pop()
                    #print(stack.isempty(),"//")
                elif precedence[stack.list[-1]] < precedence[re[i]]:
                    #print(re[i],"8")
                    stack.push(re[i])
                else:
                    while True:
                        newRegx += stack.pop()
                        if stack.isempty():
                            break
                    stack.push(re[i])
    while not stack.isempty():
        newRegx += stack.pop()
    return newRegx


class Stack:
    def __init__(self):
        self.list = []
    def pop(self):
        if len(self.list)>0:
            return self.list.pop()
    def push(self, value):
        self.list.append(value)

    def isempty(self):
        if len(self.list)==0:
            return True
        else:
            return False

def syntaxTree(x):
    synList = []
    count = 1
    for i in x:
        if i.isalpha() or i == "#":
            synList.append((i, count))
            count += 1
        else:
            synList.append(i)
    #print(synList,"s0")
    count = 1
    nodes = {}
    while len(synList) > 1:
        for i in range(len(synList)):
            if synList[i] in [".", "|"]:
                nodes[(synList[i], count)] = [[synList[i-2], synList[i-1]]]
                synList = synList[:i-2] + [(synList[i], count)] + synList[i+1:]
                #print(synList,"s1")
                count += 1
                break
            elif synList[i] == "*":
                nodes[(synList[i], count)] = [[synList[i-1]]]
                synList = synList[:i-1] + [(synList[i], count)] + synList[i+1:]
                #print(synList,"s2")
                count += 1
                break
    print(nodes)
    return nodes, synList[0]

def nullable(tree, node):
    EPSILON = ""
    if node not in tree:
        return False
    elif node  in tree:
        if node[0] =="|":
            left=nullable(tree, tree[node][0][0])
            right=nullable(tree, tree[node][0][1])
            tree[node].append(left or right)
            return left or right
        elif node[0] ==".":
            left=nullable(tree, tree[node][0][0])
            right=nullable(tree, tree[node][0][1])
            tree[node].append(left and right)
            return left and right
        elif node[0] =="*":
            nullable(tree, tree[node][0][0])
            tree[node].append(True)
            return True
class null:
    def __init__(self,n):
        self.isnull=n
c=0
c1=0
c2=0
x=[]
def isNullable(tree, node):
    copyTree=tree.copy()
    EPSILON = ""
    if node not in tree:

        return False
    elif node  in tree:
        if node[0] =="|":
            left=isNullable(tree, tree[node][0][0])
            right=isNullable(tree, tree[node][0][1])
            # if c==0:
            #     tree[node].append(left or right)
            #     c+=1
            x.append(left or right)
            return left or right
        elif node[0] ==".":
            left=isNullable(tree, tree[node][0][0])
            right=isNullable(tree, tree[node][0][1])
            # if c1==0:
            #     tree[node].append(left and right)
            #     c1+=1
            x.append(left and right)
            return left and right
        elif node[0] =="*":
            # if c2==0:
            #     nullable(tree, tree[node][0][0])
            #     tree[node].append(True)
            #     c2+=1
            x.append(True)
            return True

def firstpos(tree, node):
    if node not in tree:
        return {node[1]}
    else:
        fpos = set()
        if node[0] == "|":
            left = firstpos(tree, tree[node][0][0])
            right = firstpos(tree, tree[node][0][1])
            fpos.update(left)
            fpos.update(right)
            tree[node].append(fpos)
            return fpos
        elif node[0] == ".":
            left = firstpos(tree, tree[node][0][0])
            leftNull = isNullable(tree, tree[node][0][0])
            right = firstpos(tree, tree[node][0][1])
            if leftNull == True:
                fpos.update(left)
                fpos.update(right)
            else:
                fpos.update(left)
            tree[node].append(fpos)
            return fpos
        else:
            fpos.update(firstpos(tree, tree[node][0][0]))
            tree[node].append(fpos)
            return fpos
def lastpos(tree, node):
    if node not in tree:
        return {node[1]}
    else:
        if node[0] == "|":
            left = lastpos(tree, tree[node][0][0])
            right = lastpos(tree, tree[node][0][1])
            Lpos = left.union(right)
            tree[node].append(Lpos)
            return Lpos
        elif node[0] == ".":
            left = lastpos(tree, tree[node][0][0])
            right = lastpos(tree, tree[node][0][1])
            rightNull = isNullable(tree, tree[node][0][1])
            if rightNull == True:
                Lpos = left.union(right)
                tree[node].append(Lpos)
                return Lpos
            else:
                Lpos = right
                tree[node].append(Lpos)
                return Lpos
        else:
            Lpos = lastpos(tree, tree[node][0][0])
            tree[node].append(Lpos)
            return Lpos
def printInfo(tree):
    i=0
    for nodes in tree:
        null = x[i]
        firstpos = tree[nodes][-2]
        lastpos = tree[nodes][-1]
        print(f"Node: {nodes[0]} , Nullable: {null} , Firstpos: {firstpos} , Lastpos: {lastpos}")
        i+=1
####################################################


def followpos(tree, node, follow_dict):
    if node not in tree:
        return
    else:
        if node[0] == ".":
            left = tree[node][0][0]
            right = tree[node][0][1]
            for pos in lastpos(tree, left):
                if pos in follow_dict:
                    follow_dict[pos] = follow_dict[pos].union(firstpos(tree, right))
                else:
                    follow_dict[pos] = firstpos(tree, right)
        elif node[0] == "*":
            for pos in lastpos(tree, tree[node][0][0]):
                if pos in follow_dict:
                    follow_dict[pos] = follow_dict[pos].union(firstpos(tree, tree[node][0][0]))
                else:
                    follow_dict[pos] = firstpos(tree, tree[node][0][0])

        followpos(tree, tree[node][0][0], follow_dict)
        if node[0] in [".", "|"]:
            followpos(tree, tree[node][0][1], follow_dict)

def print_followpos(tree, follow_dict):
    print(follow_dict,"follow")
    for key in sorted(follow_dict.keys()):
        print(f"Node: {key}, Followpos: {follow_dict[key]}")
    print(f"Node: #, Followpos: {set()}")

def get_symbol_to_pos_map(tree):
    symbol_to_pos_map = {}
    for node in tree:
        if node not in tree:
            symbol = node[0]
            pos = node[1]
            if symbol not in symbol_to_pos_map:
                symbol_to_pos_map[symbol] = {pos}
            else:
                symbol_to_pos_map[symbol].add(pos)
    return symbol_to_pos_map

def construct_dfa(initial_state, follow_dict, symbol_to_pos_map):
    dfa = {}
    unmarked_states = [initial_state]
    marked_states = []

    while unmarked_states:
        current_state = unmarked_states.pop()
        marked_states.append(current_state)
        dfa[current_state] = {}

        for symbol, positions in symbol_to_pos_map.items():
            if symbol == "#":
                continue
            new_state = set()
            for pos in positions:
                if pos in current_state:
                    new_state = new_state.union(follow_dict.get(pos, set()))
            if new_state:
                dfa[current_state][symbol] = new_state
                if new_state not in marked_states and new_state not in unmarked_states:
                    unmarked_states.append(new_state)

    return dfa
def match(regex, string):
    # Construct the DFA from the regex
    dfa = construct_dfa(regex)

    # Initialize the current state to the start state of the DFA
    current_state = dfa.start_state

    # Simulate the DFA on the input string
    for c in string:
        # If there is a transition from the current state on the character, follow that transition and update the current state
        if c in dfa.transitions[current_state]:
            current_state = dfa.transitions[current_state][c]
        else:
            # Otherwise, the input string does not match the regex, return False
            return False

    # If the current state is an accept state, the input string matches the regex, return True. Otherwise, the input string does not match the regex, return False
    return current_state in dfa.accept_states



###################
regex = "(a|b)*.a.b.b"
#regex="a.a.(a|b)*.b"
regex =augmentation(regex)
regex = updateRegex(regex)
tree, lastNode = syntaxTree(regex)
print(tree,"4894")
null = isNullable(tree, lastNode)
first = firstpos(tree, lastNode)
last = lastpos(tree, lastNode)
printInfo(tree)
print(tree,"44")
##################################
follow_dict = {}
followpos(tree, lastNode, follow_dict)
print_followpos(tree, follow_dict)

symbol_to_pos_map = get_symbol_to_pos_map(tree)
initial_state = first
dfa = construct_dfa(initial_state, follow_dict, symbol_to_pos_map)

